# Classical numerics and operator learning for multidimensional gradient independent heat PDEs


# Setup

We consider the semilinear heat PDE d dimensions:
$$
    \partial_t u (t, x)
=
    \nu (\Delta_{x} u)(t, x) + f(u(t, x)),
$$
for $(t, x) \in [0,T] \times [0, S]^d$ with periodic boundary conditions.


We want to approximate the map
$$
\Phi(u(0, \cdot)) = u(T, \cdot).
$$

Problem parameters:  $T, S, \nu \in (0,\infty)$, $f \colon \mathbb{R} \to \mathbb{R}$, and distribution of initial value.

In [ ]:
import sys
import time
from scipy.fft import fft, ifft, fftfreq, fftn, ifftn
from scipy import sparse
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from ipywidgets import interact, IntSlider
import pandas as pd
import importlib
import torch
import openpyxl
import os
from datetime import datetime
import json
import seaborn as sns
sns.set_style("white")

sys.path.insert(1, '../1_Modules')

# Importing the modules
import random_function_generators
import ode_methods
import training
import training_samples_generators
import operator_learning_models
import utils
import semilinear_heat_multi_d_classical_methods
import evaluation_utils
import PDE_operations
import documentation_utils

In [ ]:
# Reloading the modules
importlib.reload(random_function_generators)
importlib.reload(ode_methods)
importlib.reload(training)
importlib.reload(training_samples_generators)
importlib.reload(utils)
importlib.reload(operator_learning_models)
importlib.reload(semilinear_heat_multi_d_classical_methods)
importlib.reload(evaluation_utils)
importlib.reload(PDE_operations)
importlib.reload(documentation_utils)


from random_function_generators import *
from ode_methods import *
from training import *
from training_samples_generators import *
from operator_learning_models import *
from utils import *
from semilinear_heat_multi_d_classical_methods import *
from evaluation_utils import *
from PDE_operations import *
from documentation_utils import *

In [ ]:
test_run = True

#Problem setup
###################################################
T = 3.
space_size = 1.
laplace_factor = 0.002
nonlin = lambda x :  - x * x * x + x
nonlin_name = "AllenCahn"
dim = 1

# initial value
var = 5000
decay_rate = 2
offset = np.power(var, 1/decay_rate)
inner_decay = 1.
start_var = 0.2
initial_value_generator = RandnFourierSeriesGeneratorStartControl([var, decay_rate, offset, inner_decay, space_size, start_var, dim])

In [ ]:
#Discretization operations
x_values = x_values_periodic
reduce_dimension = lambda values, space_resolution_step: reduce_dimension_periodic(values, space_resolution_step, dim=dim)
get_higher_nr_spacediscr = get_higher_nr_spacediscr_periodic
create_boundary_values = create_boundary_values_periodic

In [ ]:
# Name of the PDE
pde_name = f"Semilinear_heat_{dim}-dimensional_T_{T}_space_size_{space_size}_laplace_factor_{laplace_factor}_nonlin_{nonlin_name}_var_{var}_decay_rate_{decay_rate}_offset_{offset}_inner_decay_{inner_decay}_start_var_{start_var}"

#Create folder for all outputs
output_folder_dir = create_output_folder(pde_name)

# Prepare df to store data
methods_data = pd.DataFrame(columns=["nr_params", "training_time", "test_time", "L2_error", "done_trainsteps", "learning_rate_history", "batch_size_history"])
methods = {}

# Set random seeds
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

### Create Train and Test sets

In [ ]:
generate_data = True 
data_load_folder = f"Z Outputs/ZZ 2023-11-21 08h55m16s {pde_name}/"

#Nr of input points allowed to be used by methods
nr_spacediscr = 64 if test_run else 64

#Method for reference solutions for training of models
reference_algorithm = lambda initial_values, nr_timesteps: periodic_semilinear_pde_spectral_lirk(initial_values, T, laplace_factor, nonlin, space_size, nr_timesteps, dim=dim)

# Train set parameters 
# TODO: Choose something suitable here in dependence of the dimension
train_space_resolution_step = 2 if test_run else 2
train_nr_timesteps = 1000 if test_run else 1000
nr_train_samples = 1 if test_run else (2**10 if dim==2 else 2**18)
nr_validation_samples = 1 if test_run else (2**9 if dim == 2 else 2**14)

test_space_resolution_step = 4 if test_run else 4
test_nr_timesteps = 1400 if test_run else 1400
nr_test_samples = 1 if test_run else (2**9 if dim == 2 else 2**14)

only_save_rough = True

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

parameters = {
    'T': T,
    'space_size': space_size,
    'laplace_factor': laplace_factor,
    'var': var,
    'dim': dim,
    'decay_rate': decay_rate,
    'offset': offset,
    'inner_decay': inner_decay,
    'nr_spacediscr': nr_spacediscr,
    'train_space_resolution_step': train_space_resolution_step,
    'train_nr_timesteps': train_nr_timesteps,
    'nr_train_samples': nr_train_samples,
    'nr_validation_samples': nr_validation_samples,
    'test_space_resolution_step': test_space_resolution_step,
    'nr_test_samples': nr_test_samples,
    'test_nr_timesteps': test_nr_timesteps,
    'reference_algorithm': reference_algorithm.__name__,
    'only_save_rough': only_save_rough
}

# save parametesr
with open(output_folder_dir + 'train_test_parameters.json', 'w') as fp:
    json.dump(parameters, fp)

In [ ]:
# Produce train and test data
train_nr_spacediscr = get_higher_nr_spacediscr(nr_spacediscr, train_space_resolution_step)
test_nr_spacediscr = get_higher_nr_spacediscr(nr_spacediscr, test_space_resolution_step)

print("Generating train samples")
train_initial_values_fine, train_ref_sol_fine, train_initial_values_rough, train_ref_sol_rough = (
    get_data(
        initial_value_generator, reference_algorithm, 
        nr_train_samples, train_nr_spacediscr, train_nr_timesteps, 
        reduce_dimension, train_space_resolution_step, 'train', 
        output_folder_dir, generate_data, data_load_folder, parameters, only_save_rough
    ))
training_samples_generator = TrainingSamplesGeneratorFromSolutions(train_initial_values_rough, train_ref_sol_rough)

print("Generating validation samples")
validation_initial_values_fine, validation_ref_sol_fine, validation_initial_values_rough, validation_ref_sol_rough = (
    get_data(
        initial_value_generator, reference_algorithm, 
        nr_validation_samples, test_nr_spacediscr, test_nr_timesteps, 
        reduce_dimension, test_space_resolution_step, 'validate', 
        output_folder_dir, generate_data, data_load_folder, parameters, only_save_rough
    ))

print("Generating test samples")
test_initial_values_fine, test_ref_sol_fine, test_initial_values_rough, test_ref_sol_rough = (
    get_data(
        initial_value_generator, reference_algorithm, 
        nr_test_samples, test_nr_spacediscr, test_nr_timesteps, 
        reduce_dimension, test_space_resolution_step, 'test', 
        output_folder_dir, generate_data, data_load_folder, parameters, only_save_rough
    ))

In [ ]:
# Plot some reference solutions
plot_reference_solutions(train_initial_values_rough, train_ref_sol_rough, 1, dim, x_values, space_size, pde_name, output_folder_dir)

# Create models and methods to be tested

In [ ]:
# Optimizer
optimizer_class = torch.optim.Adam
# Loss function
loss_fn = torch.nn.MSELoss()

### Operator learning models

In [ ]:
# TODO: Check if the training hyperparameters are suitable for all dimensions 

# Training hyperparams
OL_training_kwargs = {
    "max_trainsteps": 1 if test_run else 100000,
    "initial_batchsize": 2**6 if test_run else 2**7 if dim ==1 else 2**6,
    "max_batchsize": 2**6 if test_run else 2**7 if dim ==1 else 2**6,
    "output_steps": 400 if test_run else 400,
    "eval_steps": 400 if test_run else 400,
    "improvement_tolerance": 0.97 if test_run else 0.97,
    "initial_lr": 0.001
}
nr_runs = 1 if test_run else (1 if dim == 2 else 3)

In [ ]:
#ANN models
ann_foldername = output_folder_dir + "Results_ANN"

#ANN Parameters
input_dim = nr_spacediscr**dim
if dim == 1:
    list_ann_params = [
        [[input_dim, 2**8, 2**9, 2**8, input_dim]],
        [[input_dim,  2**10, 2**12, 2**10, input_dim]],
        [[input_dim, 2**10, 2**12, 2**13, 2**12, 2**10, input_dim]]
    ]
if dim > 1:
    list_ann_params = [
        [[input_dim, input_dim, input_dim, input_dim]],
        [[input_dim, input_dim, 2 * input_dim, input_dim, input_dim]],
        [[input_dim, input_dim, 4 * input_dim, 4 * input_dim, input_dim, input_dim]]
    ]

create_and_train_models(
    modelclass = ANNModel, 
    list_params = list_ann_params,
    training_samples_generator = training_samples_generator,
    optimizer_class = optimizer_class,
    loss_fn = loss_fn,
    training_kwargs=OL_training_kwargs,
    lr_search_params=None,
    nr_runs = nr_runs,
    methods = methods,
    methods_data = methods_data,
    foldername = ann_foldername,
    test_input_values = test_initial_values_rough,
    test_ref_sol = test_ref_sol_rough,
    validation_input_values = validation_initial_values_rough,
    validation_ref_sol = validation_ref_sol_rough,
    pde_name = pde_name,
    local_learning_rates=False
)

In [ ]:
# CNN perdiodic models
CNNPeriodic_foldername = output_folder_dir + "Results_Periodic CNN"

#CNN Parameters [channel_dims, kernel_sizes, dim]
if dim == 1:
    list_cnn_periodic_params = [
        [[1, 50, 50, 1],[31, 31, 31], 1],
        [[1, 50, 50, 50, 1],[21, 21, 21, 21], 1],
        [[1, 50, 100, 100, 50, 1], [21, 21, 21, 21, 21], 1],
    ]
if dim == 3:
    list_cnn_periodic_params = [
        [[1, 10, 10, 10, 10, 10, 10, 10, 10, 1],[3, 3, 3, 3, 3, 3, 3, 3], dim],
        [[1, 20, 20, 20, 20, 20, 20, 20, 20, 1],[3, 3, 3, 3, 3, 3, 3, 3], dim],
        [[1, 20, 20, 20, 20, 20, 20, 20, 20, 1],[3, 3, 5, 5, 5, 5, 3, 3], dim]
    ]
if dim == 2:
    list_cnn_periodic_params = [
        [[1, 10, 10, 10, 10, 10, 10, 10, 10, 1],[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5], dim],
        [[1, 10, 10, 10, 10, 10, 10, 10, 10, 1],[5, 5, 5, 5, 9, 11, 9, 5, 5, 5, 5], dim],
        [[1, 15, 15, 15, 15, 15, 15, 15, 15, 1],[5, 5, 5, 5, 9, 11, 9, 5, 5, 5, 5], dim]
    ]


create_and_train_models(
    modelclass = CNNPeriodicnDModel, 
    list_params = list_cnn_periodic_params,
    training_samples_generator = training_samples_generator,
    optimizer_class = optimizer_class,
    loss_fn = loss_fn,
    training_kwargs=OL_training_kwargs,
    lr_search_params=None,
    nr_runs = nr_runs,
    methods = methods,
    methods_data = methods_data,
    foldername = CNNPeriodic_foldername,
    test_input_values = test_initial_values_rough,
    test_ref_sol = test_ref_sol_rough,
    validation_input_values = validation_initial_values_rough,
    validation_ref_sol = validation_ref_sol_rough,
    pde_name = pde_name,
    local_learning_rates=False
)

In [ ]:
# CNN simple enc dec models
CNNEncDec_foldername = output_folder_dir + "Results_Enc-Dec CNN"

#CNN Parameters [channel_dims, kernel_sizes, dim]
if dim == 1:
    list_cnn_enc_dec_params = [
        # [[1, 16, 32, 64],[2, 4, 2], 1],
        [[1, 4, 16, 32, 128], [4, 2, 2, 2], 1],
        [[1, 4, 16, 32, 128, 256], [4, 2, 2, 2, 2], 1],
        [[1, 8, 16, 32, 64, 128, 256],[2, 2, 2, 2, 2, 2], 1]
    ]
if dim == 3:
    list_cnn_enc_dec_params = [
        [[1, 100, 200, 400],[4, 4, 1], dim],
        [[1, 100, 200, 400, 800], [2, 2, 4, 1], dim],
        [[1, 100, 200, 400, 800, 1600], [2, 2, 2, 2, 1], dim],
    ]
if dim == 2:
    list_cnn_enc_dec_params = [
        [[1, 2**7, 2**8, 2**9, 2**10],[4, 2, 2, 1], dim],
        [[1, 2**7, 2**8, 2**9, 2**10, 2**11], [4, 2, 2, 2, 1], dim],
        [[1, 2**7, 2**8, 2**9, 2**10, 2**11, 2**12, 2**12], [2, 2, 2, 2, 2, 2, 1], dim],
        
    ]

create_and_train_models(
    modelclass = CNNEncDec, 
    list_params = list_cnn_enc_dec_params,
    training_samples_generator = training_samples_generator,
    optimizer_class = optimizer_class,
    loss_fn = loss_fn,
    training_kwargs=OL_training_kwargs,
    lr_search_params=None,
    nr_runs = nr_runs,
    methods = methods,
    methods_data = methods_data,
    foldername = CNNEncDec_foldername,
    test_input_values = test_initial_values_rough,
    test_ref_sol = test_ref_sol_rough,
    validation_input_values = validation_initial_values_rough,
    validation_ref_sol = validation_ref_sol_rough,
    pde_name = pde_name,
    local_learning_rates=False
)

In [ ]:
# FNO models
FNO_foldername = output_folder_dir + "Results_FNO"

#list is [#modes, width, depth]
list_fno_params = [
    [min(16, nr_spacediscr), 20, 4, dim],
    [min(16, nr_spacediscr), 30, 4, dim],
    [min(16, nr_spacediscr), 30, 5, dim]
]

create_and_train_models(
    modelclass = FNOnDModel, 
    list_params = list_fno_params,
    training_samples_generator = training_samples_generator,
    optimizer_class = optimizer_class,
    loss_fn = loss_fn,
    training_kwargs=OL_training_kwargs,
    lr_search_params=None,
    nr_runs = nr_runs,
    methods = methods,
    methods_data = methods_data,
    foldername = FNO_foldername,
    test_input_values = test_initial_values_rough,
    test_ref_sol = test_ref_sol_rough,
    validation_input_values = validation_initial_values_rough,
    validation_ref_sol = validation_ref_sol_rough,
    pde_name = pde_name,
    local_learning_rates=False
)

In [ ]:
# DeepONets
DeepONet_foldername = output_folder_dir + "Results_DeepONet"

eval_points = x_values(nr_spacediscr, space_size, dim)
eval_points = [np.expand_dims(eval_points_dim, axis=dim) for eval_points_dim in eval_points]
eval_points = torch.tensor(np.concatenate(eval_points, axis=dim))
DeepONet_space_grid = eval_points.reshape(-1,dim)  # Need shape (nr_spacediscr, dim)

# DeepOnet Parameters [trunk_architecture, branch_architecture, eval_points]
if dim==1:
    list_deeponet_params = [
        [[nr_spacediscr**dim, 2**11, 2**10, 2**9, 2**8],[dim, 2**7, 2**8], DeepONet_space_grid],
        [[nr_spacediscr**dim, 2**11, 2**11, 2**10, 2**9],[dim, 2**7, 2**8, 2**9], DeepONet_space_grid],
        [[nr_spacediscr**dim, 2**12, 2**13, 2**12, 2**12, 2**11],[dim, 2**9, 2**10, 2**11], DeepONet_space_grid]
    ]
if dim==2:
    list_deeponet_params = [
        [[nr_spacediscr**dim, 2**11, 2**10, 2**9, 2**8],[dim, 2**7, 2**8], DeepONet_space_grid],
        [[nr_spacediscr**dim, 2**11, 2**11, 2**10, 2**9],[dim, 2**7, 2**8, 2**9], DeepONet_space_grid],
        [[nr_spacediscr**dim, 2**12, 2**13, 2**12, 2**12, 2**11],[dim, 2**9, 2**10, 2**11], DeepONet_space_grid]
    ]
if dim==3:
        list_deeponet_params = [
        [[nr_spacediscr**dim, int(nr_spacediscr**dim/2), int(nr_spacediscr**dim/4), 100],[dim, 50, 100], DeepONet_space_grid],
        [[nr_spacediscr**dim, int(nr_spacediscr**dim/2), int(nr_spacediscr**dim/4), 500],[dim, 200, 350, 500], DeepONet_space_grid],
        [[nr_spacediscr**dim, nr_spacediscr**dim, nr_spacediscr**dim, nr_spacediscr**dim, int(nr_spacediscr**dim/2)],[dim, 500, 1000, int(nr_spacediscr**dim/2)], DeepONet_space_grid]
    ]    

create_and_train_models(
    modelclass = DeepONet, 
    list_params = list_deeponet_params,
    training_samples_generator = training_samples_generator,
    optimizer_class = optimizer_class,
    loss_fn = loss_fn,
    training_kwargs=OL_training_kwargs,
    lr_search_params=None,
    nr_runs = nr_runs,
    methods = methods,
    methods_data = methods_data,
    foldername = DeepONet_foldername,
    test_input_values = test_initial_values_rough,
    test_ref_sol = test_ref_sol_rough,
    validation_input_values = validation_initial_values_rough,
    validation_ref_sol = validation_ref_sol_rough,
    pde_name = pde_name,
    local_learning_rates=False
)

### Classical Methods

In [ ]:
 # Discretization parameters
nr_timesteps_fdm = [3, 5, 10, 15, 20, 25, 30] if test_run else ([70, 80, 90] if dim==3 else([20, 25, 30] if dim==2 else [4, 8, 16]))
nr_timesteps_spe = [70, 80, 90] if dim==3 else([20, 25, 30] if dim==2 else [8, 16, 24])

# Create FDM methods
# if dim <= 2:
#     for nr_timesteps in nr_timesteps_fdm:
#         name = f"FDM ({nr_timesteps} timesteps)"
#         methods[name] = lambda initial_values_rough, nr_timesteps=nr_timesteps: periodic_semilinear_pde_fdm_lirk(initial_values_rough, T, laplace_factor, nonlin, space_size, nr_timesteps, dim=dim)
#         methods_data.at[name, "training_time"] = 0


#Create all methods for the correponding timesteps
for nr_timesteps in nr_timesteps_spe:
    name = f"Spectral ({nr_timesteps} Crank Nicolson timesteps)"
    methods[name] = lambda initial_values_rough, nr_timesteps=nr_timesteps: periodic_semilinear_pde_spectral_lirk(initial_values_rough, T, laplace_factor, nonlin, space_size, nr_timesteps, dim=dim)
    methods_data.at[name, "training_time"] = 0

# Evaluation

In [ ]:
# Evaluate all the methods and create plots
nr_of_eval_runs = 100 if test_run else 1000
plot_histogram = False if test_run else True

# method_categories = ["ANN", "DeepONet", "Spectral"]
method_categories = ["ANN", "Periodic CNN", "Enc.-Dec. CNN", "FNO", "DeepONet", "Spectral"] + (["FDM"] if dim<=2 else [])
space_grid = x_values(nr_spacediscr, space_size, dim=dim)

evaluate_and_plot(methods, 
                  methods_data, 
                  method_categories, 
                  test_initial_values_rough, 
                  test_ref_sol_rough, 
                  space_grid, 
                  space_size, 
                  output_folder_dir, 
                  pde_name, 
                  dim=dim, 
                  nr_of_eval_runs=nr_of_eval_runs, 
                  plot_histogram=plot_histogram,
                  legend_loc=None,
                  nr_of_plots=1 if test_run else 3
                  )

#Save all the data in an Excel sheet
local_vars = locals()
params_dict = {k: [v] for k, v in local_vars.items() if isinstance(v, (int, str, float)) and k[0] != '_'}
save_excel_sheet(methods_data, params_dict, output_folder_dir + f"Results_{pde_name}.xlsx")

In [ ]:
create_error_vs_comptime_plot(method_categories, output_folder_dir, pde_name)